In [ ]:
#import libraries
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import skimage 
from skimage import io

## Import image

In [ ]:
img = io.imread('dm3.jpg')
plt.imshow(img)

## Image restoration

These steps are to restore the image to a decent quality before it is fed to the CNN for auto colorisation. 
Steps are as follows:
- Denoising
- Edge detection
- Edge enhancement
- Inpainting

### Denoising
Remove the noise in the image using the denoising function for colored images.

In [ ]:
dst = cv.fastNlMeansDenoisingColored(img,None,7,10,6,21)

fig, axes = plt.subplots(ncols = 2, figsize = (10,5))

axes[0].imshow(img)
axes[0].axis('off')

axes[1].imshow(dst)
axes[1].axis('off')

### Edge detection
Sometimes old images have cracks or fold lines on them. After removing noise, edge detection should help detect these fold lines that are white and that run all across the image. The edge detection algorithm used here is Canny edge detection. 

In [ ]:
edges = cv.Canny(dst,200,250)
plt.imshow(edges)

### Edge Enhancement

This step is carried out the enhance any strong edges or cracks present in the image. These enhanced edge lines are further used as guiding points or as a mask for our inpainting function. 

In order to enhance such folds or cracks in the picture we use opening followed by 2x dilation.

In [ ]:
kernel = np.ones((5,5),np.uint8)
dilation = cv.morphologyEx(edges, cv.MORPH_DILATE, kernel)
closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
erode = cv.morphologyEx(closing,cv.MORPH_ERODE, kernel)


fig, axes = plt.subplots(ncols = 3, figsize = (10,5))

axes[0].imshow(dilation)
axes[0].axis('off')

axes[1].imshow(closing)
axes[1].axis('off')

axes[2].imshow(erode)
axes[2].axis('off')

### Inpainting

The inpainting function uses a mask where missing areas are highlighted against a dark background. It then 'paints' any mixel pixels with a normalized value of all the surrounding pixels. The inpainting function is fed the mask from the previous image. 

In [ ]:
dest = cv.inpaint(dst,erode,6000,cv.INPAINT_TELEA)

In [ ]:
dest5 = cv.inpaint(dst,erode,5,cv.INPAINT_NS)

### Before vs after

In [ ]:
def img_restoration(img):
    dst = cv.fastNlMeansDenoisingColored(img,None,7,10,6,21)
    edges = cv.Canny(dst,200,250)
    kernel = np.ones((5,5),np.uint8)
    dilation = cv.morphologyEx(edges, cv.MORPH_DILATE, kernel)
    closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
    erode = cv.morphologyEx(closing,cv.MORPH_ERODE, kernel)
    dest5 = cv.inpaint(dst,erode,5,cv.INPAINT_NS)
    fig, axes = plt.subplots(ncols = 3, figsize = (10,5))
    axes[0].imshow(img)
    axes[0].axis('off')

    axes[1].imshow(dst)
    axes[1].axis('off')
    
    axes[2].imshow(dest5)
    axes[2].axis('off')


def process_images(img_list):
    for item in img_list:
        img = io.imread(item)
        img_restoration(img)

imgs = ['dm1.jpg','dm3.jpg','dm4.jpg','dm5.jpg','dm6.jpg','dm7.jpg','dm8.jpg','dm9.jpg','dm10.jpg','dm11.jpg','dm12.jpg']
#imgs = ['dm1.jpg','dm3.jpg']
process_images(imgs)

In [ ]:
def img_restoration2(img):
    dst = cv.fastNlMeansDenoisingColored(img,None,7,10,6,21)
    edges = cv.Canny(dst,200,250)
    kernel = np.ones((5,5),np.uint8)
    dilation = cv.morphologyEx(edges, cv.MORPH_DILATE, kernel)
    closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
    erode = cv.morphologyEx(closing,cv.MORPH_ERODE, kernel)
    dest5 = cv.inpaint(dst,closing,5,cv.INPAINT_NS)
    alpha = 0.5
    final = dest5.copy()
    #cv.addWeighted(dest5, alpha, final, 1 - alpha,0, final)
    cv.addWeighted(dst, alpha, final, 1 - alpha,0, final)
    fig, axes = plt.subplots(ncols = 4, figsize = (15,5))
    axes[0].imshow(img)
    axes[0].axis('off')

    axes[1].imshow(dst)
    axes[1].axis('off')
    
    axes[2].imshow(dest5)
    axes[2].axis('off')
    
    axes[3].imshow(final)
    #axes[3].imshow(bg2)
    axes[3].axis('off')

In [ ]:
def img_restoration3(img):
    dst = cv.fastNlMeansDenoisingColored(img,None,7,10,6,21)
    edges = cv.Canny(dst,200,250)
    kernel = np.ones((5,5),np.uint8)
    dilation = cv.morphologyEx(edges, cv.MORPH_DILATE, kernel)
    closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
    #erode = cv.morphologyEx(closing,cv.MORPH_ERODE, kernel)
    dest5 = cv.inpaint(dst,closing,5,cv.INPAINT_NS)
    #alpha = 0.7
    final = dest5.copy()
    #cv.addWeighted(dest5, alpha, final, 1 - alpha,0, final)
    #cv.addWeighted(dst, alpha, final, 1 - alpha,0, final)
    #final2 = cv.bitwise_and(final,final,mask = dest5)
    img2gray = cv.cvtColor(dst,cv.COLOR_BGR2GRAY)
    ret, mask = cv.threshold(img2gray, 100, 255, cv.THRESH_BINARY_INV)
    bg1 = cv.bitwise_and(dst,dst,mask = mask)
    mask_inv = cv.bitwise_not(mask)
    bg2 = cv.bitwise_and(final,final,mask = mask_inv)
    final2 = cv.add(bg1,bg2)
    fig, axes = plt.subplots(ncols = 4, figsize = (15,5))
    axes[0].imshow(img)
    axes[0].axis('off')

    axes[1].imshow(dst)
    axes[1].axis('off')
    
    axes[2].imshow(dest5)
    axes[2].axis('off')
    
    axes[3].imshow(final2)
    #axes[3].imshow(bg2)
    axes[3].axis('off')

In [ ]:
def process_images2(img_list):
    for item in img_list:
        img = io.imread(item)
        img_restoration2(img)

In [ ]:
img = io.imread('dm1.jpg')
img_restoration2(img)

In [ ]:
imgs = ['dm1.jpg','dm3.jpg','dm4.jpg','dm5.jpg','dm6.jpg','dm7.jpg','dm8.jpg','dm9.jpg','dm10.jpg','dm11.jpg','dm12.jpg']
#imgs = ['dm1.jpg','dm3.jpg']
process_images2(imgs)